In [1]:
from datasets import load_dataset
from deep_translator import GoogleTranslator

In [2]:
raw_dataset = load_dataset("go_emotions", name="raw")
simplified_dataset = load_dataset("go_emotions", name="simplified")

translator = GoogleTranslator(source="en", target="ru")


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/211225 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/43410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

# Translate

In [3]:
def translate_samples(samples):
    original_text = samples["text"]
    translated_batch = translator.translate_batch(original_text)

    # If the input text does not cointain any words, like ":(" or ":)"
    # translator will return None. So, I replace Nones with original text
    # as you cannont translate emojis and other stuff like this.
    for i in range(len(translated_batch)):
        if not translated_batch[i]:
            translated_batch[i] = original_text[i]
            print(f"Replaced {original_text[i]} vs {translated_batch[i]}")

    samples["ru_text"] = translated_batch
    return samples


In [4]:
ru_simplified_dataset = simplified_dataset.map(
    translate_samples, batched=True, batch_size=500
)

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Replaced :^( vs :^(
Replaced :(( vs :((
Replaced (:) vs (:)


Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

RequestError: Request exception can happen due to an api connection error. Please check your connection and try again

In [ ]:
ru_raw_dataset = raw_dataset.map(translate_samples, batched=True, batch_size=500)

# Reorder columns

In [ ]:
features = ru_simplified_dataset["train"].features.copy()
columns = list(features.to_dict().keys())
columns.remove("ru_text")
for column in ["ru_text"] + columns:
    features[column] = features.pop(column)
ru_simplified_dataset_final = ru_simplified_dataset.cast(features)

In [ ]:
features = ru_raw_dataset["train"].features.copy()
columns = list(features.to_dict().keys())
columns.remove("ru_text")
for column in ["ru_text"] + columns:
    features[column] = features.pop(column)
ru_raw_dataset_final = ru_raw_dataset.cast(features)

# Create .csv

In [ ]:
for split in ["train", "validation", "test"]:
    ru_simplified_dataset_final[split].to_csv(f"ru-go-emotions-simplified-{split}.csv")
ru_raw_dataset_final["train"].to_csv("ru-go-emotions-raw.csv")


# Push to Hugging Face

In [ ]:
ru_simplified_dataset_final.push_to_hub("ru_go_emotions", config_name="simplified")
ru_raw_dataset_final.push_to_hub("ru_go_emotions", config_name="raw")